# Visualization of Neighborhoods Tutorial

<div class="alert alert-block alert-info">
<b> Objective:</b> 
This tutorial covers how to perform visualize neighborhoods based on two approaches.  
    
    1) Grabbing a bounding box region a vertex
    2) Grabbing n neighbors around a vertex

In [1]:
import napari
%gui qt5

In [2]:
from brainlit.viz.swc import *
from brainlit.utils.ngl_pipeline import NeuroglancerSession

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]
Using TensorFlow backend.


**Reading data from s3 path**

In [7]:
s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
seg_id,  v_id, mip = 2, 10, 1 # skeleton/neuron id, index/row of df, resolution quality
df = read_s3(s3_path, seg_id, mip)
df.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00, 24.74it/s]
/Users/jennytrieu/Desktop/NDD/brainlit/brainlit/viz/swc.py:46: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  delim_whitespace=True,


,sample,structure,x,y,z,r,parent
0,1,0,4713.0,4470.0,3857.0,1.0,-1
1,4,192,4721.0,4445.0,3849.0,1.0,1
2,7,64,4723.0,4446.0,3851.0,1.0,4
3,8,0,4728.0,4449.0,3852.0,1.0,7
4,14,0,4746.0,4445.0,3858.0,1.0,8


**Converting dataframe to graph data structure to understand how vertices are connected**

In [8]:
G = df_to_graph(df)
paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

The graph was decomposed into 179 paths


**Plotting the entire skeleton/neuron**

In [9]:
viewer = napari.Viewer(ndisplay=3)
# it is important that the number of paths put into 'data=' is at the most 1024
viewer.add_points(data=np.concatenate(paths)[804:], edge_width=2, edge_color='white', name='Skeleton 2')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', name='Skeleton 2')

<Shapes layer 'Skeleton 2 [1]' at 0x133013690>

<div class="alert alert-block alert-warning">

# Bounding Box Method

<div>

**Creating a bounding box based on a particular vertex of interest in order to get a group of neurons neighboring the vertex of interest**

In [10]:
# Create an NGL session to get the bounding box
ngl_sess = NeuroglancerSession(mip = 1)
img, bbbox, vox = ngl_sess.pull_chunk(seg_id, v_id, 1, 1, 1)
bbox = bbbox.to_list()
box = (bbox[:3], bbox[3:])

Downloading:   0%|          | 0/3 [00:00<?, ?it/s]


**Getting all the coordinates of the group surrounding the vertex of interest using get_sub_neuron()**  
Note: data correction step necessary due to recentering in function!

In [13]:
G_sub = get_sub_neuron(G, box)

# preventing the re-centring of nodes to the bounding box corner (origin of the new coordinate frame)
for id in list(G_sub.nodes):
    G_sub.nodes[id]["x"] = G_sub.nodes[id]["x"] + box[0][0]
    G_sub.nodes[id]["y"] = G_sub.nodes[id]["y"] + box[0][1]
    G_sub.nodes[id]["z"] = G_sub.nodes[id]["z"] + box[0][2]
    
paths_sub = graph_to_paths(G_sub)

**Plotting vertex and vertex neighborhood**

In [15]:
# grab the coordinates of the vertex from the skeleon 
cv_skel = CloudVolume(s3_path, mip=mip)
skel = cv_skel.skeleton.get(seg_id)
vertex = skel.vertices[v_id]/cv_skel.scales[mip]["resolution"]
print(vertex)

viewer.add_points(data=np.concatenate(paths_sub), edge_width=2, edge_color='blue', name='Skeleton 2')
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='Neighborhood',edge_width=5)

# display vertex
viewer.add_points(data=np.array(vertex), edge_width=5, edge_color='green', name='vertex')

Downloading: 100%|██████████| 1/1 [00:00<00:00, 19.27it/s]


[6258.6576087  6582.55139803 2211.12398785]


<Points layer 'vertex' at 0x17b302490>

![Boundary Method](assets/boundary_method.png "Boundary Method")

<div class="alert alert-block alert-warning">

# Neighbors Method

<div>

In [17]:
# grab the coordinates of the vertex from the skeleon 
cv_skel = CloudVolume(s3_path, mip=mip)
skel = cv_skel.skeleton.get(seg_id)
vertex = skel.vertices[v_id]/cv_skel.scales[mip]["resolution"]
print(vertex)

# figure out where the vertex information is stored in the dataframe returned by read_s3
x, y, z = np.round((vertex))[0], np.round((vertex))[1], np.round((vertex))[2]
slice_df = (df[(df.x == x)&(df.y==y)&(df.z==z)])
v_idx = np.where((df.x == x)&(df.y==y)&(df.z==z))
v_idx = v_idx[0][0]
print(v_idx)
slice_df.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00, 20.44it/s]

[6258.6576087  6582.55139803 2211.12398785]
469


,sample,structure,x,y,z,r,parent
469,434,0,6259.0,6583.0,2211.0,1.0,431


**On another napari window, plot again the entire neuron/skeleton.**

In [18]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(data=np.concatenate(paths, axis=0)[1024:], edge_width=2, edge_color='white', name='all_points')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', edge_width=3, name='skeleton')

<Shapes layer 'skeleton' at 0x17eab0590>

**Get the coordinates of the neighobrs around vertex of interest using get_bfs_subgraph() and graphs_to_paths**

In [19]:
v_id_pos = v_idx  # the row index/number of the data frame
depth = 10  # the depth up to which the graph must be constructed

G_bfs=get_bfs_subgraph(G, v_id_pos, depth, df=df)  # perform Breadth first search to obtain a graph of interest
paths_bfs = graph_to_paths(G=G_bfs[0])  # obtain all the paths for visualization purposes

**Plot the vertex and vertex neighborhood**

In [20]:
x,y,z = df.iloc[v_id_pos]['x'], df.iloc[v_id_pos]['y'], df.iloc[v_id_pos]['z']

# display vertex
viewer.add_points(data=np.array([x,y,z]), edge_width=5, edge_color='orange', name='bfs_vertex')

# display all neighbors around vertex
viewer.add_points(data=np.concatenate(paths_bfs), edge_color='red', edge_width=2, name='bfs_points')
viewer.add_shapes(data=paths_bfs, shape_type='path', edge_color='red', edge_width=3, name='bfs_sub_skeleton')

<Shapes layer 'bfs_sub_skeleton' at 0x17b52c050>

![BFS Method](assets/bfs_method.png "BFS method")

<div class="alert alert-block alert-warning">

# Visualizing the output of both methods overlaid

<div>

**Create new napari window**

In [21]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(data=np.concatenate(paths, axis=0)[1024:], edge_width=2, edge_color='white', name='all_points')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', edge_width=3, name='full_skeleton')

<Shapes layer 'full_skeleton' at 0x186a1e2d0>

**Plot vertices and neighborhoods of each method on the same napari window to compare method outputs**

In [22]:
# display vertex of the boundary method
viewer.add_points(data=np.array(vertex), edge_width=5, edge_color='green', name='boundary_vertex')

# display all neighbors around vertex of boundary method
viewer.add_points(data=np.concatenate(paths_sub), edge_width=2, edge_color='blue', name='boundary_skeleton_pts')
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='boundary_skeleton_lines',edge_width=5)

# display vertex of the bfs method
x,y,z = df.iloc[v_id_pos]['x'], df.iloc[v_id_pos]['y'], df.iloc[v_id_pos]['z']
viewer.add_points(data=np.array([x,y,z]), edge_width=5, edge_color='orange', name='bfs_vertex')

# display all neighbors around vertex of bfs method
viewer.add_points(data=np.concatenate(paths_bfs), edge_color='red', edge_width=2, name='bfs_skeleton_pts')
viewer.add_shapes(data=paths_bfs, shape_type='path', edge_color='red', edge_width=3, name='bfs_skeleton_lines')

<Shapes layer 'bfs_skeleton_lines' at 0x18351db50>

![](overlay.gif)